In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits

In [3]:
hdu1_data = hdulist[1].data
df_c = pd.DataFrame(hdu1_data.tolist(), columns=hdu1_data.names)

In [4]:
# keep 10 rows TODO : REMOVE for the real pipeline
df_c = df_c.head(10)

In [32]:
df = df_c # used a s backup since df_c takes 5 minutes to load

In [33]:
# drop 1st row (calibration)
df = df.drop(0)

# subset all chemical abundances (collumn name contain "_FE")
chemical_subset = df.filter(regex='_FE', axis=1) 

# merge with + df.filter(regex='_H', axis=1)
chemical_subset = pd.concat([chemical_subset, df.filter(regex='_H', axis=1)], axis=1)

to_remove_from_chemicals = ['RV_FEH', 'MIN_H', 'MAX_H', 'GAIAEDR3_R_HI_GEO', 'GAIAEDR3_R_HI_PHOTOGEO','CU_FE_ERR',
                            'P_FE_ERR','P_FE_FLAG',
                            'CU_FE_FLAG',
                            'M_H', 'M_H_ERR', 'X_H_SPEC', 'X_H', 'X_H_ERR']
chemical_subset = chemical_subset.drop(to_remove_from_chemicals, axis=1)

# create a df without the chemical abundances
df = df.drop(chemical_subset.columns, axis=1)

to_remove_from_main_dataset = ['P_FE_ERR',
'P_FE_FLAG',
'CU_FE_ERR', 'GAIAEDR3_PARALLAX_ERROR', 'GAIAEDR3_PMRA', 'GAIAEDR3_PMRA_ERROR',
'GAIAEDR3_PMDEC', 'GAIAEDR3_PMDEC_ERROR', 'GAIAEDR3_PHOT_G_MEAN_MAG',
'GAIAEDR3_PHOT_BP_MEAN_MAG', 'GAIAEDR3_PHOT_RP_MEAN_MAG',
'GAIAEDR3_DR2_RADIAL_VELOCITY', 'GAIAEDR3_DR2_RADIAL_VELOCITY_ERROR',
'GAIAEDR3_R_MED_GEO', 'GAIAEDR3_R_LO_GEO', 'GAIAEDR3_R_HI_GEO',
'GAIAEDR3_R_MED_PHOTOGEO', 'GAIAEDR3_R_LO_PHOTOGEO',
'GAIAEDR3_R_HI_PHOTOGEO','MEANFIB', 'SIGFIB','AK_TARG', 'AK_TARG_METHOD',
'APOGEE_TARGET1', 'APOGEE_TARGET2', 'APOGEE2_TARGET1',
'APOGEE2_TARGET2', 'APOGEE2_TARGET3', 'APOGEE2_TARGET4',
'RV_CHI2','RV_CCFWHM', 'RV_AUTOFWHM','VSCATTER', 'VERR', 'RV_FEH', 'RV_FLAG', 'MIN_H', 'MAX_H',
'MIN_JK', 'MAX_JK', 'GAIAEDR3_SOURCE_ID', 'GAIAEDR3_PARALLAX',
'ASPCAP_GRID', 'ASPCAP_CHI2','FRAC_BADPIX','FRAC_LOWSNR', 'FRAC_SIGSKY',
'ELEM_CHI2', 'ELEMFRAC', 'EXTRATARG', 'MEMBERFLAG', 'MEMBER','X_H_SPEC', 'X_M_SPEC',
'FRAC_BADPIX','FRAC_LOWSNR', 'FRAC_SIGSKY','ELEM_CHI2', 'ELEMFRAC',
'X_H_SPEC', 'X_M_SPEC','TEFF_SPEC', 'LOGG_SPEC','CU_FE_FLAG', 'ALT_ID', 'PROGRAMNAME', 
'RV_TEFF', 'RV_LOGG', 'RV_ALPHA', 'RV_CARB', 'SNREV', 'SFD_EBV', 'N_COMPONENTS']

df = df.drop(to_remove_from_main_dataset, axis=1)


chemical_subset = chemical_subset[chemical_subset.columns.drop(list(chemical_subset.filter(regex='_SPEC')))]

# remove collumns with 100% NaN values
chemical_subset = chemical_subset.dropna(axis=1, how='all')

chemical_subset_err = chemical_subset.filter(regex='_ERR', axis=1) 
chemical_subset_flag = chemical_subset.filter(regex='_FLAG', axis=1) 

# remove _ERR and _FLAG from chemical_subset
chemical_subset = chemical_subset.drop(chemical_subset_err.columns, axis=1)
chemical_subset = chemical_subset.drop(chemical_subset_flag.columns, axis=1)


# parse the list of all chemical elements in the dataset

chemical_elements = [x.split('_')[0] for x in chemical_subset.columns]

In [ ]:
display(chemical_subset.columns)
display(chemical_subset_err.columns)
display(chemical_subset_flag.columns)

Index(['APOGEE_ID', 'TELESCOPE', 'FIELD', 'RA', 'DEC', 'GLON', 'GLAT', 'J',
       'J_ERR', 'H', 'H_ERR', 'K', 'K_ERR', 'AK_WISE', 'TARGFLAGS', 'SURVEY',
       'NVISITS', 'SNR', 'STARFLAG', 'STARFLAGS', 'ANDFLAG', 'ANDFLAGS',
       'VHELIO_AVG', 'PARAMFLAG', 'ASPCAPFLAG', 'ASPCAPFLAGS', 'X_H',
       'X_H_ERR', 'X_M', 'X_M_ERR', 'TEFF', 'TEFF_ERR', 'LOGG', 'LOGG_ERR',
       'M_H', 'M_H_ERR', 'ALPHA_M', 'ALPHA_M_ERR', 'VMICRO', 'VMACRO',
       'VSINI'],
      dtype='object')

Index(['C_FE', 'CI_FE', 'N_FE', 'O_FE', 'NA_FE', 'MG_FE', 'AL_FE', 'SI_FE',
       'S_FE', 'K_FE', 'CA_FE', 'TI_FE', 'TIII_FE', 'V_FE', 'CR_FE', 'MN_FE',
       'CO_FE', 'NI_FE', 'CE_FE', 'FE_H'],
      dtype='object')

Index(['C_FE_ERR', 'CI_FE_ERR', 'N_FE_ERR', 'O_FE_ERR', 'NA_FE_ERR',
       'MG_FE_ERR', 'AL_FE_ERR', 'SI_FE_ERR', 'S_FE_ERR', 'K_FE_ERR',
       'CA_FE_ERR', 'TI_FE_ERR', 'TIII_FE_ERR', 'V_FE_ERR', 'CR_FE_ERR',
       'MN_FE_ERR', 'CO_FE_ERR', 'NI_FE_ERR', 'CE_FE_ERR', 'FE_H_ERR'],
      dtype='object')

Index(['C_FE_FLAG', 'CI_FE_FLAG', 'N_FE_FLAG', 'O_FE_FLAG', 'NA_FE_FLAG',
       'MG_FE_FLAG', 'AL_FE_FLAG', 'SI_FE_FLAG', 'S_FE_FLAG', 'K_FE_FLAG',
       'CA_FE_FLAG', 'TI_FE_FLAG', 'TIII_FE_FLAG', 'V_FE_FLAG', 'CR_FE_FLAG',
       'MN_FE_FLAG', 'CO_FE_FLAG', 'NI_FE_FLAG', 'CE_FE_FLAG', 'FE_H_FLAG'],
      dtype='object')

In [46]:
for col in df.columns:
    print(col)

APOGEE_ID
TELESCOPE
FIELD
RA
DEC
GLON
GLAT
J
J_ERR
H
H_ERR
K
K_ERR
AK_TARG
AK_TARG_METHOD
AK_WISE
TARGFLAGS
SURVEY
NVISITS
SNR
STARFLAG
STARFLAGS
ANDFLAG
ANDFLAGS
VHELIO_AVG
VSCATTER
VERR
RV_FEH
MIN_H
MAX_H
MIN_JK
MAX_JK
GAIAEDR3_SOURCE_ID
GAIAEDR3_PARALLAX
GAIAEDR3_PARALLAX_ERROR
GAIAEDR3_PMRA
GAIAEDR3_PMRA_ERROR
GAIAEDR3_PMDEC
GAIAEDR3_PMDEC_ERROR
GAIAEDR3_PHOT_G_MEAN_MAG
GAIAEDR3_PHOT_BP_MEAN_MAG
GAIAEDR3_PHOT_RP_MEAN_MAG
GAIAEDR3_DR2_RADIAL_VELOCITY
GAIAEDR3_DR2_RADIAL_VELOCITY_ERROR
GAIAEDR3_R_MED_GEO
GAIAEDR3_R_LO_GEO
GAIAEDR3_R_HI_GEO
GAIAEDR3_R_MED_PHOTOGEO
GAIAEDR3_R_LO_PHOTOGEO
GAIAEDR3_R_HI_PHOTOGEO
PARAMFLAG
ASPCAPFLAGS
X_H
X_H_ERR
ELEM_CHI2
ELEMFRAC
MEMBERFLAG
MEMBER
X_H_SPEC
TEFF
TEFF_ERR
LOGG
LOGG_ERR
M_H
M_H_ERR
ALPHA_M
ALPHA_M_ERR
VMICRO
VMACRO
VSINI
TEFF_SPEC
LOGG_SPEC


In [47]:
df['TARGFLAGS']

1         APOGEE2_TWOBIN_GT_0_8,APOGEE2_WISE_DERED,APOGE...
2         APOGEE2_SFD_DERED,APOGEE2_SHORT,APOGEE2_NORMAL...
3                           APOGEE_NO_DERED,APOGEE_TELLURIC
4                                              APOGEE_SHORT
5           APOGEE2_MAGCLOUD_CANDIDATE,APOGEE2_GAIA_OVERLAP
                                ...                        
733896    APOGEE2_SFD_DERED,APOGEE2_SHORT,APOGEE2_NORMAL...
733897                                         APOGEE_SHORT
733898    APOGEE2_WISE_DERED,APOGEE2_SHORT,APOGEE2_MANGA...
733899      APOGEE2_MAGCLOUD_CANDIDATE,APOGEE2_GAIA_OVERLAP
733900                      APOGEE_NO_DERED,APOGEE_TELLURIC
Name: TARGFLAGS, Length: 733900, dtype: object

In [9]:
targflags=df['TARGFLAGS'].str.split(",")

In [10]:
targflags_expl=targflags.explode()

In [11]:
targflags_expl

1              APOGEE2_TWOBIN_GT_0_8
1                 APOGEE2_WISE_DERED
1                      APOGEE2_SHORT
1              APOGEE2_NORMAL_SAMPLE
2                  APOGEE2_SFD_DERED
                     ...            
733898         APOGEE2_ONEBIN_GT_0_3
733899    APOGEE2_MAGCLOUD_CANDIDATE
733899          APOGEE2_GAIA_OVERLAP
733900               APOGEE_NO_DERED
733900               APOGEE_TELLURIC
Name: TARGFLAGS, Length: 2070005, dtype: object

In [12]:
starflags=df['STARFLAGS'].str.split(",")

In [13]:
starflags_expl=starflags.explode()

In [14]:
starflags_expl

1             PERSIST_MED
1             PERSIST_LOW
2                        
3            PERSIST_HIGH
4                        
               ...       
733898    BRIGHT_NEIGHBOR
733898        PERSIST_LOW
733899                   
733900    BRIGHT_NEIGHBOR
733900       PERSIST_HIGH
Name: STARFLAGS, Length: 899845, dtype: object

In [15]:
andflags=df['ANDFLAGS'].str.split(",")

In [16]:
andflags_expl=andflags.explode()

In [17]:
andflags_expl

1                     
2                     
3         PERSIST_HIGH
4                     
5                     
              ...     
733896                
733897                
733898                
733899                
733900                
Name: ANDFLAGS, Length: 779168, dtype: object

In [18]:
paramflags_expl=df["PARAMFLAG"].explode()

In [19]:
paramflags_expl

1                0
1         33554432
1                0
1                0
1                0
            ...   
733900          32
733900         288
733900           0
733900           0
733900           0
Name: PARAMFLAG, Length: 6605100, dtype: object

In [20]:
aspcapflags=df["ASPCAPFLAGS"].str.split(",")

In [21]:
aspcapflags_expl=aspcapflags.explode()

In [22]:
aspcapflags_expl

1                         
2              VMICRO_WARN
2                 N_M_WARN
3                         
4                         
                ...       
733896    MULTIPLE_SUSPECT
733897                    
733898                    
733899                    
733900            N_M_WARN
Name: ASPCAPFLAGS, Length: 1338362, dtype: object

In [23]:
df["MEMBERFLAG"].unique()
#pas besoin de split/explode

array([                  0,      70368744177664,              131072,
             1099511627776, 1152921504606846976,       2199023255552,
        576460752303423488,     140737488355328,             1048576,
             4398046511104,          1073741824,               16384,
                     32768,                4096, 2305843009213693952,
                      8192,               65536,              262144,
            17592186044416,  288230376151711744,          2147483648,
                        16,             4194304,            33554432,
                         4,            16777216,         17179869184,
                       128,  144115188075855872,                   8,
         72057594037927936,        137438953472,                 256,
             8796093022208,                1024,                  64,
                       512,         68719476736,          4294967296,
                         1,   36028797018963968,     562949953421312,
                  67

In [24]:
survey=df["SURVEY"].str.split(",")

In [25]:
survey_expl=survey.explode()

In [26]:
survey_expl

1               apogee2
2               apogee2
3                apogee
4                apogee
5              apogee2s
              ...      
733896          apogee2
733897           apogee
733898    manga-apogee2
733899         apogee2s
733900           apogee
Name: SURVEY, Length: 745357, dtype: object

In [27]:
for col in chemical_subset_flag.columns:
    print(col,chemical_subset_flag[col].dtypes)

C_FE_FLAG int64
CI_FE_FLAG int64
N_FE_FLAG int64
O_FE_FLAG int64
NA_FE_FLAG int64
MG_FE_FLAG int64
AL_FE_FLAG int64
SI_FE_FLAG int64
S_FE_FLAG int64
K_FE_FLAG int64
CA_FE_FLAG int64
TI_FE_FLAG int64
TIII_FE_FLAG int64
V_FE_FLAG int64
CR_FE_FLAG int64
MN_FE_FLAG int64
CO_FE_FLAG int64
NI_FE_FLAG int64
CE_FE_FLAG int64
FE_H_FLAG int64


In [48]:
chemical_subset_flag["AL_FE_FLAG"]

1         64
2          0
3          0
4          0
5          0
          ..
733896     0
733897     0
733898     0
733899     0
733900     0
Name: AL_FE_FLAG, Length: 733900, dtype: int64

In [17]:
# Données pour la table
chemicals_flag_bitmask = {
    "Bit Name": [
        "GRIDEDGE_BAD", "CALRANGE_BAD", "OTHER_BAD", "FERRE_FAIL", 
        "PARAM_MISMATCH_BAD", "FERRE_ERR_USED", "TEFF_CUT", 
        "GRIDEDGE_WARN", "CALRANGE_WARN", "OTHER_WARN", "FERRE_WARN", 
        "PARAM_MISMATCH_WARN", "OPTICAL_WARN", "ERR_WARN", 
        "FAINT_WARN", "PARAM_FIXED", "RV_WARN", 
        "SPEC_RC", "SPEC_RGB", "LOGG_CAL_MS", "LOGG_CAL_RGB_MS"
    ],
    "Binary Digit": [
        0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 24, 25, 26, 27
    ],
    "Description": [
        "Parameter within 1/8 grid spacing of grid edge : true value may be off grid",
        "Parameter outside valid range of calibration determination",
        "Other error condition",
        "Failed solution in FERRE",
        "Elemental abundance from window differs significantly from parameter abundance",
        "FERRE uncertainty used (larger than parametric uncertainty)",
        "Star in region of parameter space where abundances do not appear valid for this element",
        "Parameter within 1 grid spacing of grid edge (not necessarily bad)",
        "Parameter in possibly unreliable range of calibration determination",
        "Other warning condition",
        "FERRE warning (not implemented?)",
        "Elemental abundance from window differs from parameter abundance",
        "Comparison with optical abundances suggests problem",
        "Large expected uncertainty or upper limit based on location in parameter space (Teff, [M/H], S/N)",
        "Warning based on faint star/RV combination",
        "Parameter set at fixed value, not fit",
        "RV puts important line off of chip",
        "Spectroscopically identified RC star",
        "Spectroscopically identified RGB star",
        "DR16: use MS gravity calibration",
        "DR16: use RBG/MS transition gravity calibration"
    ]
}

# Création du DataFrame
chemicals_flag_bitmask_df = pd.DataFrame(chemicals_flag_bitmask)

# Afficher le tableau
print(chemicals_flag_bitmask_df)

               Bit Name  Binary Digit  \
0          GRIDEDGE_BAD             0   
1          CALRANGE_BAD             1   
2             OTHER_BAD             2   
3            FERRE_FAIL             3   
4    PARAM_MISMATCH_BAD             4   
5        FERRE_ERR_USED             5   
6              TEFF_CUT             6   
7         GRIDEDGE_WARN             8   
8         CALRANGE_WARN             9   
9            OTHER_WARN            10   
10           FERRE_WARN            11   
11  PARAM_MISMATCH_WARN            12   
12         OPTICAL_WARN            13   
13             ERR_WARN            14   
14           FAINT_WARN            15   
15          PARAM_FIXED            16   
16              RV_WARN            17   
17              SPEC_RC            24   
18             SPEC_RGB            25   
19          LOGG_CAL_MS            26   
20      LOGG_CAL_RGB_MS            27   

                                          Description  
0   Parameter within 1/8 grid spacing of 

In [ ]:
# Fonction pour obtenir la décomposition en puissances de 2
def decomposition_en_puissances(n):
    puissances = []
    for i in range(n.bit_length()):  # bit_length donne le nombre de bits nécessaires
        if n & (1 << i):  # Vérifie si le ième bit est à 1
            puissances.append(i)  # Ajoute la puissance correspondante
    return ",".join(map(str, puissances[::-1]))  # Inverser pour afficher du plus grand au plus petit

# Appliquer la fonction à toutes les colonnes du DataFrame
for col in chemical_subset_flag.columns:
    chemical_subset_flag[col] = chemical_subset_flag[col].apply(decomposition_en_puissances)

In [74]:
chemical_subset_flag["N_FE_FLAG"]

1           0
2         256
3          32
4         288
5           0
         ... 
733896      0
733897      0
733898      0
733899    288
733900    288
Name: N_FE_FLAG, Length: 733900, dtype: int64

In [51]:
# Fonction pour convertir un nombre en liste de puissances de 2
def decompose_to_powers_of_2(number):
    powers = []
    bit_position = 0
    while number > 0:
        if number & 1:
            powers.append(bit_position)
        number >>= 1
        bit_position += 1
    return powers

In [88]:
# Appliquer cette fonction sur chaque valeur du dataframe
df_transformed = chemical_subset_flag.applymap(decompose_to_powers_of_2)

# Affichage du DataFrame transformé
print(df_transformed)

C:\Users\axioz\AppData\Local\Temp\ipykernel_11332\1054887720.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformed = chemical_subset_flag.applymap(decompose_to_powers_of_2)


       C_FE_FLAG CI_FE_FLAG N_FE_FLAG O_FE_FLAG NA_FE_FLAG MG_FE_FLAG  \
1             []         []        []        []         []         []   
2             []         []       [8]        []         []         []   
3             []         []       [5]        []         []         []   
4             []         []    [5, 8]        []         []         []   
5             []        [0]        []        []         []         []   
...          ...        ...       ...       ...        ...        ...   
733896        []         []        []        []         []         []   
733897        []         []        []        []         []         []   
733898        []         []        []        []         []         []   
733899        []         []    [5, 8]        []         []         []   
733900        []         []    [5, 8]        []         []         []   

       AL_FE_FLAG SI_FE_FLAG S_FE_FLAG K_FE_FLAG CA_FE_FLAG TI_FE_FLAG  \
1             [6]         []       [6]        [] 

In [89]:
df_transformed["N_FE_FLAG"]

1             []
2            [8]
3            [5]
4         [5, 8]
5             []
           ...  
733896        []
733897        []
733898        []
733899    [5, 8]
733900    [5, 8]
Name: N_FE_FLAG, Length: 733900, dtype: object

In [90]:
# Ajouter une colonne 'ID' au DataFrame
df_transformed['ID'] = range(1, len(df_transformed) + 1)

In [ ]:
# Pour chaque colonne contenant des listes, appliquez explode
for col in df_transformed.columns:
    if df_transformed[col].apply(lambda x: isinstance(x, list)).any():  # Vérifie que la colonne contient des listes
        df_transformed = df_transformed.explode(col, ignore_index=True)

In [1]:
df_transformed

NameError: name 'df_exploded' is not defined